In [14]:
#%%
# Import các thư viện cần thiết
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

# Khởi tạo SummaryWriter cho TensorBoard
writer = SummaryWriter('runs/cifar10_resnet18_experiment')
print("Đã khởi tạo SummaryWriter cho TensorBoard.")

Đã khởi tạo SummaryWriter cho TensorBoard.


In [15]:
# Tải mô hình ResNet-18 đã được huấn luyện trước
print("Đang tải mô hình ResNet-18 đã được huấn luyện trước...")
# pretrained=True: Tải trọng số đã được huấn luyện trên bộ ImageNet lớn
resnet18 = torchvision.models.resnet18(pretrained=True)
print("Đã tải xong mô hình ResNet-18.")

# Điều chỉnh lớp cuối cùng của mô hình cho 10 lớp đầu ra (CIFAR-10)
# resnet18.fc: Lớp cuối cùng (fully connected) của ResNet-18
num_ftrs = resnet18.fc.in_features # Số lượng đặc trưng đầu vào của lớp cuối
resnet18.fc = nn.Linear(num_ftrs, 10) # Thay thế lớp cuối bằng một lớp Linear mới có 10 đầu ra (cho CIFAR-10)
print(f"Đã thay đổi lớp cuối cùng của ResNet-18 để có {10} lớp đầu ra.")

# Xác định thiết bị tính toán (CPU hoặc GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device) # Chuyển mô hình sang thiết bị đã chọn (GPU hoặc CPU)
print(f"Mô hình sẽ được huấn luyện trên: {device}")

# Đóng băng tất cả các tham số (trọng số) của mô hình pretrained
for param in resnet18.parameters():
    param.requires_grad = False
# Chỉ cho phép lớp fc mới được cập nhật trọng số (huấn luyện)
resnet18.fc.weight.requires_grad = True
resnet18.fc.bias.requires_grad = True
print("Đã đóng băng các lớp pretrained và chỉ huấn luyện lớp đầu ra mới.")

Đang tải mô hình ResNet-18 đã được huấn luyện trước...
Đã tải xong mô hình ResNet-18.
Đã thay đổi lớp cuối cùng của ResNet-18 để có 10 lớp đầu ra.
Mô hình sẽ được huấn luyện trên: cuda:0
Đã đóng băng các lớp pretrained và chỉ huấn luyện lớp đầu ra mới.


In [16]:

# Định nghĩa các phép biến đổi dữ liệu
transform = transforms.Compose([
    transforms.ToTensor(), # Chuyển ảnh sang Tensor PyTorch
    # Chuẩn hóa ảnh với giá trị trung bình (mean) và độ lệch chuẩn (std) cho 3 kênh màu
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# ... existing code ...

# Tải và chuẩn bị tập dữ liệu CIFAR-10
print("Đang tải và chuẩn bị tập dữ liệu CIFAR-10...")
# Sửa 'root='./DL_Practice/data'' thành 'root='./data''
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Sửa 'root='./DL_Practice/data'' thành 'root='./data''
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
print("Đã chuẩn bị xong tập dữ liệu CIFAR-10.")
# Tên các lớp trong CIFAR-10 (để hiển thị trực quan)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Đang tải và chuẩn bị tập dữ liệu CIFAR-10...
Đã chuẩn bị xong tập dữ liệu CIFAR-10.


In [17]:

# Trực quan hóa ảnh mẫu và đồ thị mô hình trong TensorBoard
dataiter = iter(trainloader) # Tạo iterator từ DataLoader
images, labels = next(dataiter) # Lấy một lô ảnh và nhãn

# ---- THÊM DÒNG NÀY ĐỂ CHUYỂN ẢNH SANG CÙNG THIẾT BỊ VỚI MÔ HÌNH ----
images = images.to(device)
# --------------------------------------------------------------------

# Ghi đồ thị mô hình vào TensorBoard
writer.add_graph(resnet18, images) # resnet18 là mô hình, images là đầu vào mẫu
print("Đã ghi đồ thị mô hình vào TensorBoard.")

# Hiển thị một lưới ảnh mẫu vào TensorBoard
img_grid = torchvision.utils.make_grid(images) # Tạo lưới ảnh từ lô ảnh
writer.add_image('CIFAR10_images', img_grid) # Ghi lưới ảnh vào TensorBoard
print("Đã ghi ảnh mẫu vào TensorBoard.")

Đã ghi đồ thị mô hình vào TensorBoard.
Đã ghi ảnh mẫu vào TensorBoard.


In [18]:
#%%
# Định nghĩa hàm mất mát và bộ tối ưu hóa
criterion = nn.CrossEntropyLoss() # Hàm mất mát Cross-Entropy, phù hợp cho phân loại
# optimizer: SGD (Stochastic Gradient Descent)
# Chỉ tối ưu hóa các tham số mà require_grad=True (lớp fc mới)
# lr=0.001: Tốc độ học
# momentum=0.9: Tham số momentum giúp tăng tốc hội tụ
optimizer = optim.SGD(filter(lambda p: p.requires_grad, resnet18.parameters()), lr=0.001, momentum=0.9)
print("Đã định nghĩa hàm mất mát và bộ tối ưu hóa.")

Đã định nghĩa hàm mất mát và bộ tối ưu hóa.


In [19]:

# Huấn luyện mô hình
print("Bắt đầu huấn luyện mô hình...")
global_step = 0 # Biến đếm bước toàn cục cho TensorBoard
num_epochs = 2 # Số lượng epoch huấn luyện

for epoch in range(num_epochs): # Lặp qua từng epoch
    running_loss = 0.0 # Mất mát tích lũy trong epoch
    for i, data in enumerate(trainloader, 0): # Lặp qua từng lô dữ liệu
        inputs, labels = data # Lấy ảnh và nhãn từ lô

        # Chuyển dữ liệu sang thiết bị đã chọn (GPU/CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad() # Đặt gradient về 0
        outputs = resnet18(inputs) # Lan truyền tiến (dự đoán)
        loss = criterion(outputs, labels) # Tính toán mất mát
        loss.backward() # Lan truyền ngược (tính gradient)
        optimizer.step() # Cập nhật trọng số

        running_loss += loss.item() # Cộng dồn mất mát
        if i % 2000 == 1999: # Cứ sau 2000 lô thì in ra và ghi log
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            # Ghi training loss vào TensorBoard
            writer.add_scalar('training loss',
                              running_loss / 2000,
                              epoch * len(trainloader) + i)
            running_loss = 0.0 # Đặt lại mất mát tích lũy
    global_step += 1 # Tăng bước toàn cục sau mỗi epoch

print('Hoàn thành huấn luyện.')

# Tùy chọn: Lưu mô hình đã huấn luyện
# torch.save(resnet18.state_dict(), './DL_Practice/cifar10_resnet18_model.pth')
# print("Đã lưu mô hình vào './DL_Practice/cifar10_resnet18_model.pth'")

Bắt đầu huấn luyện mô hình...
[1,  2000] loss: 2.589
[1,  4000] loss: 2.618
[1,  6000] loss: 2.624
[1,  8000] loss: 2.569
[1, 10000] loss: 2.589
[1, 12000] loss: 2.605
[2,  2000] loss: 2.574
[2,  4000] loss: 2.608
[2,  6000] loss: 2.657
[2,  8000] loss: 2.566
[2, 10000] loss: 2.581
[2, 12000] loss: 2.640
Hoàn thành huấn luyện.


In [20]:
#%%
# Đánh giá mô hình trên tập kiểm tra
print("Bắt đầu đánh giá mô hình trên tập kiểm tra...")
correct = 0 # Số lượng dự đoán đúng
total = 0 # Tổng số mẫu
resnet18.eval() # Đặt mô hình ở chế độ đánh giá (tắt dropout, batchnorm...)
with torch.no_grad(): # Tắt tính toán gradient để tiết kiệm bộ nhớ và tăng tốc
    for data in testloader:
        images, labels = data

        # Chuyển dữ liệu sang thiết bị đã chọn (GPU/CPU)
        images, labels = images.to(device), labels.to(device)

        outputs = resnet18(images) # Dự đoán
        _, predicted = torch.max(outputs.data, 1) # Lấy nhãn dự đoán có xác suất cao nhất
        total += labels.size(0) # Tăng tổng số mẫu
        correct += (predicted == labels).sum().item() # Đếm số lượng dự đoán đúng

accuracy = 100 * correct / total # Tính độ chính xác
print(f'Độ chính xác của mô hình trên 10000 ảnh kiểm tra: {accuracy:.2f}%')
# Ghi độ chính xác vào TensorBoard
writer.add_scalar('Accuracy on Test Set', accuracy, num_epochs)

# Đóng SummaryWriter
writer.close()
print("Đã đóng SummaryWriter.")
print("\nĐể xem trực quan hóa TensorBoard, mở terminal (trong cùng thư mục với notebook này) và chạy:")
print("tensorboard --logdir=runs")
print("Sau đó truy cập http://localhost:6006 trong trình duyệt của bạn.")

Bắt đầu đánh giá mô hình trên tập kiểm tra...
Độ chính xác của mô hình trên 10000 ảnh kiểm tra: 25.82%
Đã đóng SummaryWriter.

Để xem trực quan hóa TensorBoard, mở terminal (trong cùng thư mục với notebook này) và chạy:
tensorboard --logdir=runs
Sau đó truy cập http://localhost:6006 trong trình duyệt của bạn.
